Verify the sobol sequence as used in the CosmoGrid

In [1]:
import os, h5py
import numpy as np

import sobol_seq
from sobol_seq import i4_sobol
from scipy.stats.qmc import Sobol
# from chaospy import create_sobol_samples

from icecream import ic

from msfm.utils import cosmogrid
from msfm.utils.input_output import read_yaml

In [2]:
cosmo_params_info = cosmogrid.get_cosmo_params_info("../../data/CosmoGridV1_original_metainfo.h5", "grid")

conf_dir = "../../configs/config.yaml"
conf = read_yaml(conf_dir)

sobol_priors = np.array(conf["analysis"]["grid"]["prior"]["sobol"])
sobol_labels = ["Om", "s8", "Ob","H0", "ns", "w0"]

### verify with respect to the CosmoGrid
Differences in `Ob` are expected due to bugs in concept

In [3]:
counter = 0
for i in range(conf["analysis"]["grid"]["n_cosmos"]):
# for i in range(len(cosmo_params_info)):
    cosmo_params = [cosmo_params_info[label][i] for label in sobol_labels]
    cosmo_params = np.array(cosmo_params, dtype=np.float32)

    i_sobol = cosmo_params_info["sobol_index"][i]

    # sobol_seq
    sobol_point, _ = i4_sobol(6, i_sobol)
    sobol_params = sobol_point*np.squeeze(np.diff(sobol_priors[:6])) + sobol_priors[:6,0]
    sobol_params = sobol_params.astype(np.float32)

    np.allclose(sobol_params, cosmo_params, rtol=1e-3, atol=1e-5)

    if not np.isclose(cosmo_params[2], sobol_params[2], rtol=1e-3, atol=1e-5):
        print(i_sobol)
        print(sobol_params)
        print(cosmo_params)
        counter += 1

print(f"There are {counter} cosmologies where the two parameter sets don't match")

2753
[ 3.0927736e-01  1.0369629e+00  5.9831545e-02  8.1898926e+01
  9.6360350e-01 -6.3620192e-01]
[ 3.0927736e-01  1.0369629e+00  5.8831543e-02  8.1898926e+01
  9.6360350e-01 -6.3620192e-01]
There are 1 cosmologies where the two parameter sets don't match
